# Exemplo de cálculo de matriz de impedância de fase e de matriz de impedância de sequência

In [19]:
import numpy as np
np.set_printoptions(precision=4)
%precision 4

u'%.4f'

![rede](Figuras/fig_021.png)

## Distância entre os condutores

In [21]:
dab = 2.5
dbc = 4.5
dca = 7.0
dan = 5.6569
dbn = 4.272
dcn = 5.0

## Características dos condutores

In [20]:
gmrf = 0.0244
rf = 0.306

In [22]:
gmrn = 0.00814
rn = 0.592

## Matriz de distâncias

In [23]:
D = np.array([[gmrf, dab, dca, dan],
              [dab, gmrf, dbc, dbn],
              [dca, dbc, gmrf, dcn],
              [dan, dbn, dcn, gmrn]])
print D

[[ 0.0244  2.5     7.      5.6569]
 [ 2.5     0.0244  4.5     4.272 ]
 [ 7.      4.5     0.0244  5.    ]
 [ 5.6569  4.272   5.      0.0081]]


## Cálculo da matriz de impedâncias primitiva

Para o cálculo das impedâncias primitivas utilizamos as equações de Carson modificadas:

\begin{eqnarray}
        \hat z_{ii} & = & r_i + 0,0953 + j \cdot 0,12134 \cdot \left ( ln \frac{1}{GMR_i} + 7,93402 \right) ~~ \Omega/milha \nonumber\\
        \hat z_{ij} & = & 0,0953 + j \cdot 0,12134 \cdot \left ( ln \frac{1}{D_{ij}} + 7,93402 \right) ~~ \Omega/milha \nonumber
\end{eqnarray}

In [24]:
zabc = list()
for i in range(4):
    zl = list()
    for j in range(4):
        if i != j:
            zf = 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
        else:
            if i == 3:
                zf = rn + 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
            else:
                zf = rf + 0.0953 + 1j * (0.12134 * (np.log(1.0/D[i,j]) + 7.93402))
        zl.append(zf)
    zabc.append(zl)
Z = np.array(zabc)
print Z

[[ 0.4013+1.4133j  0.0953+0.8515j  0.0953+0.7266j  0.0953+0.7524j]
 [ 0.0953+0.8515j  0.4013+1.4133j  0.0953+0.7802j  0.0953+0.7865j]
 [ 0.0953+0.7266j  0.0953+0.7802j  0.4013+1.4133j  0.0953+0.7674j]
 [ 0.0953+0.7524j  0.0953+0.7865j  0.0953+0.7674j  0.6873+1.5465j]]


## Particionamento da matriz de impedâncias primitiva para redução de Kron

In [25]:
zij = Z[0:3, 0:3]
zij = np.reshape(zij, (3, 3))
print zij

[[ 0.4013+1.4133j  0.0953+0.8515j  0.0953+0.7266j]
 [ 0.0953+0.8515j  0.4013+1.4133j  0.0953+0.7802j]
 [ 0.0953+0.7266j  0.0953+0.7802j  0.4013+1.4133j]]


In [26]:
zin = Z[0:3, 3]
zin = np.reshape(zin, (3, 1))
print zin

[[ 0.0953+0.7524j]
 [ 0.0953+0.7865j]
 [ 0.0953+0.7674j]]


In [27]:
znn = Z[3,3]
znn = np.reshape(znn, (1, 1))
print znn

[[ 0.6873+1.5465j]]


In [28]:
znj = Z[3, 0:3]
znj = np.reshape(znj, (1, 3))
print znj

[[ 0.0953+0.7524j  0.0953+0.7865j  0.0953+0.7674j]]


## Aplicação da redução de Kron

A equação que representa a redução de Kron é dada por:

\begin{equation}
        \mathbf{\hat z_{abc}} = (\mathbf{\hat z_{ij}} - \mathbf{\hat z_{in}} \cdot \mathbf{\hat z^{-1}_{nn}} \cdot \mathbf{\hat z_{nj}} )
\end{equation}

In [29]:
Zabc = zij - np.dot(np.dot(zin, np.linalg.inv(znn)) ,znj)
Zabc

array([[ 0.4576+1.078j ,  0.1559+0.5017j,  0.1535+0.3849j],
       [ 0.1559+0.5017j,  0.4666+1.0482j,  0.1580+0.4236j],
       [ 0.1535+0.3849j,  0.1580+0.4236j,  0.4615+1.0651j]])

## Cálculo da matriz de impedâncias de sequência

\begin{equation}
    \mathbf{Z_{012}} = \mathbf{A}^{-1} \cdot \mathbf{Z_{abc}} \cdot \mathbf{A} = 
    \left[
    \begin{array}{ccc}
             z_{00} & z_{01} & z_{02} \\
             z_{10} & z_{11} & z_{12} \\
             z_{20} & z_{21} & z_{22}
    \end{array} 
    \right]  
\end{equation}

Em que $A$:

\begin{equation}
\mathbf{A} = 
    \left[ 
    \begin{array}{ccc}
           1 & 1 & 1\\
           1 & a^2 & a\\
           1 & a & a^2
    \end{array} 
    \right]
\end{equation}

In [30]:
a = -0.5 + 1j * np.sqrt(3.0) / 2.0

In [31]:
A = np.array([[1.0, 1.0, 1.0], [1.0, a**2, a], [1.0, a, a**2]])
print A

[[ 1.0+0.j     1.0+0.j     1.0+0.j   ]
 [ 1.0+0.j    -0.5-0.866j -0.5+0.866j]
 [ 1.0+0.j    -0.5+0.866j -0.5-0.866j]]


In [32]:
Z012 = np.dot(np.linalg.inv(A), np.dot(Zabc, A))
print Z012

[[ 0.7735+1.9373j  0.0256+0.0115j -0.0321+0.0159j]
 [-0.0321+0.0159j  0.3061+0.627j  -0.0723-0.006j ]
 [ 0.0256+0.0115j  0.0723-0.0059j  0.3061+0.627j ]]


## Simulando a transposição de uma linha de distribuição

Os termos que não estão na diagonal de $\mathbf{Z_{012}}$ representam um acoplamento entre as impedâncias de sequência. 

Para que essas impedâncias mútuas fossem zero, é preciso assumir que a linha é transposta.

Nesse caso:

\begin{eqnarray}
        z_s & = & \frac{1}{3} \cdot(z_{aa} + z_{bb} + z_{cc}) \\
        z_m & = & \frac{1}{3} \cdot(z_{ab} + z_{bc} + z_{ca}) 
\end{eqnarray}

Substituindo na matriz de impedância de fase:

\begin{equation}
        \mathbf{Z_{abc}} = 
        \left[
        \begin{array}{ccc}
             z_{s} & z_{m} & z_{m} \\
             z_{m} & z_{s} & z_{m} \\
             z_{m} & z_{m} & z_{s}
        \end{array} 
        \right] ~~ \Omega/milha
\end{equation}

In [37]:
zs = 1.0/3.0 * (Zabc[0, 0] + Zabc[1, 1] + Zabc[2, 2])
print zs

(0.46188361492+1.0637522137j)


In [38]:
zm = 1.0/3.0 * (Zabc[0, 1] + Zabc[0, 2] + Zabc[1, 2])
print zm

(0.155813629382+0.436751976048j)


In [39]:
Z1abc = np.array([[zs, zm, zm], [zm, zs, zm], [zm, zm, zs]])
Z1abc

array([[ 0.4619+1.0638j,  0.1558+0.4368j,  0.1558+0.4368j],
       [ 0.1558+0.4368j,  0.4619+1.0638j,  0.1558+0.4368j],
       [ 0.1558+0.4368j,  0.1558+0.4368j,  0.4619+1.0638j]])

In [40]:
Z1012 = np.dot(np.linalg.inv(A), np.dot(Z1abc, A))
print np.round(Z1012, 4)

[[ 0.7735+1.9373j  0.0000+0.j      0.0000+0.j    ]
 [-0.0000-0.j      0.3061+0.627j   0.0000+0.j    ]
 [-0.0000+0.j      0.0000+0.j      0.3061+0.627j ]]
